In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os 
os.chdir("drive/MyDrive/torch_FCN")

In [3]:
import numpy as np
def sliding_window(signal, win_len, shift):

    n_windows = int((signal.shape[0] - win_len) / shift) + 1
    samples = []
    
    for i in range(n_windows):
        samples.append(signal[i * shift: i * shift + win_len])
    
    samples = np.stack(samples)
    
    return samples

In [4]:
from sklearn.preprocessing import MinMaxScaler
def minmaxscaling(array):
    scaler = MinMaxScaler()
    scaler.fit(array)
    result = scaler.transform(array)
    return result

In [5]:
from torch.utils.data import Dataset, DataLoader
import torch
import pickle

class WESADDataset(Dataset):

    def __init__(self, path):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        with open(path, 'rb') as file:
            trainset = pickle.load(file, encoding='latin1')

        self.ACCx = torch.tensor(trainset['ACCx'])
        self.ACCy = torch.tensor(trainset['ACCy'])
        self.ACCz = torch.tensor(trainset['ACCz'])
        self.EDA = torch.tensor(trainset['EDA'])
        self.TEMP = torch.tensor(trainset['TEMP'])
        self.BVP = torch.tensor(trainset['BVP'])
        self.Ys = torch.tensor(trainset['Ys'])
        
        
    def __len__(self):
        return len(self.ACCx)

    def __getitem__(self, idx):
        return self.ACCx[idx], self.ACCy[idx], self.ACCz[idx], self.EDA[idx], self.TEMP[idx], self.BVP[idx], self.Ys[idx]

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvLayers(nn.Module):
    def __init__(self):
        super(ConvLayers, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, 8, padding='same')
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = nn.Conv1d(128, 256, 5, padding='same')
        self.bn2 = nn.BatchNorm1d(256)
        self.conv3 = nn.Conv1d(256, 128, 3, padding='same')
        self.bn3 = nn.BatchNorm1d(128)
        self.pool = nn.AdaptiveAvgPool1d(1)
        
    def forward(self, x):
        
        x = x.float()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.pool(x).squeeze(-1)
    
class FCN_model(nn.Module):
    def __init__(self, input_channels):
        super(FCN_model, self).__init__()
        # Needs code here
        self.num_channel = input_channels
        self.convLayersList = []
        for i in range(input_channels):
            self.convLayersList.append(ConvLayers().to(device))
        self.fc = nn.Linear(128 * input_channels, 2)

    def forward(self, accx, accy, accz, eda, temp, bvp):
        # Needs code here
        accx_out = self.convLayersList[0](accx)
        accy_out = self.convLayersList[1](accy)
        accz_out = self.convLayersList[2](accz)
        eda_out = self.convLayersList[3](eda)
        temp_out = self.convLayersList[4](temp)
        bvp_out = self.convLayersList[5](bvp)
        multi_modal_out = torch.cat([accx_out, accy_out,
                                     accz_out, eda_out,
                                     temp_out, bvp_out], dim=-1)
        
        return self.fc(multi_modal_out)


In [8]:
training_set = WESADDataset('train.pkl')
validation_set = WESADDataset('test.pkl')
train_loader = DataLoader(training_set, batch_size=128,
                        shuffle=True, num_workers=2)
val_loader = DataLoader(validation_set, batch_size=128,
                        shuffle=True, num_workers=2)

In [9]:
len(training_set) + len(validation_set)

1105

In [11]:
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FCN_model(6).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train the model
for epoch in tqdm(range(20)):
    losses = []
    for accx, accy, accz, eda, temp, bvp, label in train_loader:
        accx = accx.permute(0, 2, 1).to(device)
        accy = accy.permute(0, 2, 1).to(device)
        accz = accz.permute(0, 2, 1).to(device)
        eda = eda.permute(0, 2, 1).to(device)
        temp = temp.permute(0, 2, 1).to(device)
        bvp = bvp.permute(0, 2, 1).to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        pred = model(accx, accy, accz, eda, temp, bvp)
        #print(label)
        loss = loss_fn(pred, label)
        #print(loss)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    print("\nepoch {} have loss {}".format(epoch, np.mean(losses)))
    
    with torch.no_grad():
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        for accx, accy, accz, eda, temp, bvp, label in val_loader:
            accx = accx.permute(0, 2, 1).to(device)
            accy = accy.permute(0, 2, 1).to(device)
            accz = accz.permute(0, 2, 1).to(device)
            eda = eda.permute(0, 2, 1).to(device)
            temp = temp.permute(0, 2, 1).to(device)
            bvp = bvp.permute(0, 2, 1).to(device)
            label = label.to(device)

            pred = model(accx, accy, accz, eda, temp, bvp)
            logits = pred.argmax(-1)
            TP += torch.logical_and(label == 1, logits == 1).sum().detach().cpu().numpy()
            FP += torch.logical_and(label == 0, logits == 1).sum().detach().cpu().numpy()
            TN += torch.logical_and(label == 0, logits == 0).sum().detach().cpu().numpy()
            FN += torch.logical_and(label == 1, logits == 0).sum().detach().cpu().numpy()
        
        P = TP / (TP + FP)
        R = TP / (TP + FN)
        F1 = (2 * P * R) / (P + R)

        print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN, 'F1:', F1)

  0%|          | 0/20 [00:00<?, ?it/s]


epoch 0 have loss 0.5307490229606628


  5%|▌         | 1/20 [00:05<01:38,  5.17s/it]

TP: 19 FP: 0 TN: 154 FN: 50 F1: 0.4318181818181819

epoch 1 have loss 0.359895259141922


 10%|█         | 2/20 [00:10<01:33,  5.19s/it]

...

TP: 45 FP: 15 TN: 139 FN: 24 F1: 0.6976744186046512

epoch 18 have loss 0.12819477915763855


 95%|█████████▌| 19/20 [01:40<00:05,  5.39s/it]

TP: 46 FP: 15 TN: 139 FN: 23 F1: 0.7076923076923076

epoch 19 have loss 0.12190765887498856


100%|██████████| 20/20 [01:46<00:00,  5.31s/it]

TP: 47 FP: 14 TN: 140 FN: 22 F1: 0.7230769230769231


In [25]:
len(training_set)

2318

In [27]:
len(validation_set)

556